# Create Solr Core

In [1]:
# Delete any existing core
!curl -X GET "http://localhost:8983/solr/admin/cores?action=UNLOAD&core=search_reddit&deleteInstanceDir=true&deleteDataDir=true"

{
  "responseHeader":{
    "status":0,
    "QTime":736
  }
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100    60  100    60    0     0     52      0  0:00:01  0:00:01 --:--:--    52
100    60  100    60    0     0     52      0  0:00:01  0:00:01 --:--:--    52


In [20]:
import shutil

# Create directory
shutil.copytree("C:\\solr-9.5.0-slim\\server\\solr\\configsets\\_default", "C:\\solr-9.5.0-slim\\server\\solr\\search_reddit")

'C:\\solr-9.5.0-slim\\server\\solr\\search_reddit'

In [21]:
# Create solr core with default settings
!curl -X GET "http://localhost:8983/solr/admin/cores?action=CREATE&name=search_reddit&instanceDir=search_reddit"

{
  "responseHeader":{
    "status":0,
    "QTime":271
  },
  "core":"search_reddit"
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100    86  100    86    0     0    176      0 --:--:-- --:--:-- --:--:--   176


# Solr recommends we use HTTP requests to edit schema. Because of data integrity issues

In [22]:
# Get current schema, should be default/reserved fields
!curl -X GET "http://localhost:8983/solr/search_reddit/schema/fields"

{

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   574  100   574    0     0   2714      0 --:--:-- --:--:-- --:--:--  2720



  "responseHeader":{
    "status":0,
    "QTime":0
  },
  "fields":[{
    "name":"_nest_path_",
    "type":"_nest_path_"
  },{
    "name":"_root_",
    "type":"string",
    "indexed":true,
    "stored":false,
    "docValues":false
  },{
    "name":"_text_",
    "type":"text_general",
    "multiValued":true,
    "indexed":true,
    "stored":false
  },{
    "name":"_version_",
    "type":"plong",
    "indexed":false,
    "stored":false
  },{
    "name":"id",
    "type":"string",
    "multiValued":false,
    "indexed":true,
    "required":true,
    "stored":true
  }]
}


# Add custom schema & index

https://solr.apache.org/guide/6_6/field-properties-by-use-case.html#field-properties-by-use-case
https://solr.apache.org/guide/6_6/defining-fields.html#defining-fields

In [23]:
!curl -X POST -H "Content-type:application/json" --data-binary "{"add-field":{"name":"body","type":"text_general","stored":true,"indexed":true}}" http://localhost:8983/api/cores/search_reddit/schema



{
  "responseHeader":{
    "status":0,
    "QTime":135
  }
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   126  100    60  100    66    165    182 --:--:-- --:--:-- --:--:--   349


In [24]:
# Get current schema, should be default/reserved fields
!curl -X GET "http://localhost:8983/solr/search_reddit/schema/fields"

{
  "responseHeader":{
    "status":0,
    "QTime":0
  },
  "fields":[{
    "name":"_nest_path_",
    "type":"_nest_path_"
  },{
    "name":"_root_",
    "type":"string",
    "indexed":true,
    "stored":false,
    "docValues":false
  },{
    "name":"_text_",
    "type":"text_general",
    "multiValued":true,
    "indexed":true,
    "stored":false
  },{
    "name":"_version_",
    "type":"plong",
    "indexed":false,
    "stored":false
  },{
    "name":"body",
    "type":"text_general",
    "indexed":true,
    "stored":true
  },{
    "name":"id",
    "type":"string",
    "multiValued":false,
    "indexed":true,
    "required":true,
    "stored":true
  }]
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   664  100   664    0     0   3088      0 --:--:-- --:--:-- --:--:--  3102


# Ingest data

In [27]:
# add document to solr
!curl "http://localhost:8983/solr/search_reddit/update?commit=true" --data-binary @data2/comments_subset.csv -H "Content-type:application/csv"

{
  "responseHeader":{
    "status":0,
    "QTime":283
  }
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  291k  100    60  100  290k    121   586k --:--:-- --:--:-- --:--:--  586k


# Test query data

In [32]:
!curl -X GET http://localhost:8983/solr/search_reddit/query -d "q=body:congrats&fl=id,body&rows=10"

{
  "responseHeader":{
    "status":0,
    "QTime":0,
    "params":{
      "q":"body:congrats",
      "fl":"id,body",
      "rows":"10"
    }
  },
  "response":{
    "numFound":3,
    "start":0,
    "numFoundExact":true,
    "docs":[{
      "body":["Congrats, Gav!"],
      "id":"78d9f10c-5ba8-4782-ad5b-d99fcddb1ea2"
    },{
      "body":["Congrats, you got yourself one of the (in my opinion) very few cool electric cars."],
      "id":"43a01b1e-7539-481a-9681-088bf144235d"
    },{
      "body":["congrats man, looks great ! please keep us posted on the process. I'm keen to see if I could find one for myself :)"],
      "id":"307420c4-b359-4075-b5af-dc9b5726dc1e"
    }]
  }
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   715  100   681  100    34   3167    158 --:--:-- --:--:-- --:--:--  3341
